In [1]:
%matplotlib inline

In [2]:
import keras

Using TensorFlow backend.


In [34]:
from math import ceil

In [45]:
from datetime import datetime

In [76]:
import json

In [3]:
DATA_PATH = 'data/'

In [43]:
MODEL_PATH = 'models/'

In [87]:
BATCH_SIZE = 128

## Load data

In [5]:
from keras.applications.xception import Xception

In [6]:
from keras.preprocessing import image

In [8]:
from keras.models import Model

In [9]:
from keras.layers import Dense, GlobalAveragePooling2D

In [29]:
from keras.optimizers import Adam

In [97]:
train_generator = image.ImageDataGenerator(
    rotation_range=90, 
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
)

In [ ]:
train_batches = train_generator.flow_from_directory(
    DATA_PATH + 'train/',
    target_size=(299, 299),
    shuffle=True,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
)

Found 12929 images belonging to 49 classes.


In [93]:
valid_batches = image.ImageDataGenerator().flow_from_directory(
    DATA_PATH + 'valid/',
    target_size=(299, 299),
    shuffle=True,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
)

Found 3256 images belonging to 49 classes.


In [94]:
num_classes = train_batches.num_classes

In [11]:
base_model = Xception(weights='imagenet', include_top=False)

83689472/83683744 [==============================] - 5s 0us/step


In [24]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [25]:
model = Model(inputs=base_model.input, outputs=predictions)

In [27]:
for layer in base_model.layers:
    layer.trainable = False

In [32]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [36]:
model.fit_generator(
    train_batches,
    steps_per_epoch=int(ceil(train_batches.samples/BATCH_SIZE)),
    validation_data=valid_batches,
    validation_steps=int(ceil(valid_batches.samples/BATCH_SIZE)),
    epochs=1,
)

Epoch 1/1
203/203 [==============================] - 308s 2s/step - loss: 2.7868 - acc: 0.2393 - val_loss: 2.4143 - val_acc: 0.3127


In [38]:
model.optimizer.lr = 0.01

In [39]:
model.fit_generator(
    train_batches,
    steps_per_epoch=int(ceil(train_batches.samples/BATCH_SIZE)),
    validation_data=valid_batches,
    validation_steps=int(ceil(valid_batches.samples/BATCH_SIZE)),
    epochs=8,
)

Epoch 1/8
203/203 [==============================] - 304s 1s/step - loss: 2.1944 - acc: 0.3817 - val_loss: 2.1443 - val_acc: 0.3796
Epoch 2/8
203/203 [==============================] - 307s 2s/step - loss: 1.9402 - acc: 0.4469 - val_loss: 2.0021 - val_acc: 0.4235
Epoch 3/8
203/203 [==============================] - 306s 2s/step - loss: 1.7665 - acc: 0.4988 - val_loss: 1.8995 - val_acc: 0.4487
Epoch 4/8
203/203 [==============================] - 306s 2s/step - loss: 1.6738 - acc: 0.5264 - val_loss: 1.8462 - val_acc: 0.4656
Epoch 5/8
203/203 [==============================] - 306s 2s/step - loss: 1.5581 - acc: 0.5553 - val_loss: 1.9765 - val_acc: 0.4125
Epoch 6/8
203/203 [==============================] - 305s 2s/step - loss: 1.4874 - acc: 0.5729 - val_loss: 1.7510 - val_acc: 0.4886
Epoch 7/8
203/203 [==============================] - 306s 2s/step - loss: 1.4311 - acc: 0.5985 - val_loss: 1.7070 - val_acc: 0.4957
Epoch 8/8
203/203 [==============================] - 305s 2s/step - loss: 1.

In [81]:
def save_model(model, train_batches):
    model_name = MODEL_PATH + f'carnet-{datetime.now().isoformat()}'

    model.save_weights(model_name + '-weights.h5')
    with open(model_name + '-model.json', 'w') as f:
        f.write(model.to_json())
    
    with open(model_name + '-classes.json', 'w') as f:
        json.dump(train_batches.class_indices, f)

In [56]:
model.optimizer.lr = 0.001

In [95]:
model.fit_generator(
    train_batches,
    steps_per_epoch=int(ceil(train_batches.samples/BATCH_SIZE)),
    validation_data=valid_batches,
    validation_steps=int(ceil(valid_batches.samples/BATCH_SIZE)),
    epochs=10,
)

Epoch 1/10
102/102 [==============================] - 408s 4s/step - loss: 3.6645 - acc: 0.2126 - val_loss: 2.0044 - val_acc: 0.4361
Epoch 2/10
102/102 [==============================] - 371s 4s/step - loss: 3.1947 - acc: 0.2447 - val_loss: 2.1133 - val_acc: 0.3900
Epoch 3/10
102/102 [==============================] - 373s 4s/step - loss: 3.0949 - acc: 0.2607 - val_loss: 2.1830 - val_acc: 0.3907
Epoch 4/10
102/102 [==============================] - 370s 4s/step - loss: 2.9362 - acc: 0.2651 - val_loss: 2.2174 - val_acc: 0.3649
Epoch 5/10
102/102 [==============================] - 372s 4s/step - loss: 2.8518 - acc: 0.2837 - val_loss: 2.2109 - val_acc: 0.3682
Epoch 6/10
102/102 [==============================] - 370s 4s/step - loss: 2.8265 - acc: 0.2850 - val_loss: 2.1351 - val_acc: 0.3759
Epoch 7/10
102/102 [==============================] - 368s 4s/step - loss: 2.7302 - acc: 0.2937 - val_loss: 2.1355 - val_acc: 0.3787
Epoch 8/10
102/102 [==============================] - 376s 4s/step - 

In [96]:
save_model(model, train_batches)

In [ ]:
model.fit_generator(
    train_batches,
    steps_per_epoch=int(ceil(train_batches.samples/BATCH_SIZE)),
    validation_data=valid_batches,
    validation_steps=int(ceil(valid_batches.samples/BATCH_SIZE)),
    epochs=20,
)

Epoch 1/20
102/102 [==============================] - 411s 4s/step - loss: 2.5271 - acc: 0.3135 - val_loss: 2.1454 - val_acc: 0.3698
Epoch 2/20
102/102 [==============================] - 372s 4s/step - loss: 2.5278 - acc: 0.3144 - val_loss: 2.1045 - val_acc: 0.3756
Epoch 3/20
102/102 [==============================] - 373s 4s/step - loss: 2.4831 - acc: 0.3222 - val_loss: 2.1263 - val_acc: 0.3759
Epoch 4/20
102/102 [==============================] - 369s 4s/step - loss: 2.4672 - acc: 0.3296 - val_loss: 2.1311 - val_acc: 0.3805
Epoch 5/20
102/102 [==============================] - 371s 4s/step - loss: 2.4595 - acc: 0.3263 - val_loss: 2.1238 - val_acc: 0.3747
Epoch 6/20
102/102 [==============================] - 371s 4s/step - loss: 2.3994 - acc: 0.3406 - val_loss: 2.0899 - val_acc: 0.3839
Epoch 7/20
102/102 [==============================] - 372s 4s/step - loss: 2.3698 - acc: 0.3460 - val_loss: 2.1197 - val_acc: 0.3793
Epoch 8/20
102/102 [==============================] - 372s 4s/step - 

In [ ]:
save_model(model, train_batches)